# Data Extractor for CMCC-ESM2 NetCDF files
## The following snippets are for Amon, Omon datasets with a specific name format

This notebook is specifically written/hardcoded for CMCC-ESM2 .nc files (Lovato et al., 2021) sourced from ESGF. The purpose was to extract timeseries data from specific nc files (i.e pr, thetao, so etc) to be used as datasets for applying the methodology I developed for selecting representative years. 

Note: These codes/snippets are hardcoded for nc files which are of Omon, Amon resolution. For Omon, it is specifically coded for variables which have 4 dimensions e.g thetao with [t,lev,i,j] dimensions or pr with [t, i, j] dimensions. The code has been tested for pr, thetao, so, chl parameters of Amon and Omon resolutions only. Please check the dimension of your netcdf file and modify the snippets accordingly in your device if you wish to use the same approach for extracting data from nc files.



In [9]:
import netCDF4 as nc
import numpy as np
import pandas as pd
from glob import glob
import os
import matplotlib.pyplot as plt

# Path to your NetCDF files
filepaths = sorted(glob('H:/M.Sc Thesis - Data, Methodology, Results/NetCDF Files/CMCC-ESM2 NC/126/pr_Amon_CMCC-ESM2_ssp126_r1i1p1f1_gn_*.nc'))
filename = os.path.basename(filepaths[0])
parts = filename.split('_')
var = parts[0]
freq = parts[1]
scenario = parts[3]
ssp = scenario[-3:]
data_type = freq.lower()

print(filename)
print(var)
print(ssp)
print(data_type)

pr_Amon_CMCC-ESM2_ssp126_r1i1p1f1_gn_201501-210012.nc
pr
126
amon


In [39]:
location_type = 'point' # 'domain'  # Input the type of data you need to export

''' 
The following snippet is for point and domain coordinates of locations. This snippet works ONLY for Amon and Omon datasets of CMCC-EMS2
'''

# Load data
if location_type == 'point':
    locations_data = pd.read_csv('pointxy.csv')   #sample coordinate file
    location_names = locations_data['Location Name']
    location_lons = locations_data['Longitude']
    location_lats = locations_data['Latitude']
    
elif location_type == 'domain':
    boundary_data = pd.read_csv('domainxy.csv')  #sample coordinate file
    boundary_lons = boundary_data['Longitude']
    boundary_lats = boundary_data['Latitude']
    lat_min, lat_max = min(boundary_lats), max(boundary_lats)
    lon_min, lon_max = min(boundary_lons), max(boundary_lons)


# Initialize data storage
time_series_data = {}

if location_type == 'point':
    for name in location_names:
        time_series_data[name] = []

    for filepath in filepaths:
        dataset = nc.Dataset(filepath, mode='r')
        if data_type == 'omon':
            lats = dataset.variables['latitude'][:]
            lons = dataset.variables['longitude'][:]
        elif data_type == 'amon':
            lats = dataset.variables['lat'][:]
            lons = dataset.variables['lon'][:]
        time = dataset.variables['time'][:]
        time_units = dataset.variables['time'].units
        time_calendar = dataset.variables['time'].calendar
        times = nc.num2date(time, units=time_units, calendar=time_calendar)
        

        # Identify node indices for the given locations
        node_indices = []
        for lat, lon in zip(location_lats, location_lons):
            if data_type == 'omon':
                distance = np.sqrt((lats - lat)**2 + (lons - lon)**2)
                node_indices.append(np.unravel_index(np.argmin(distance), lats.shape))
            elif data_type == 'amon':
                lat_idx = (np.abs(lats - lat)).argmin()
                lon_idx = (np.abs(lons - lon)).argmin()
                node_indices.append((lat_idx, lon_idx))

        for t in range(len(times)):
            for name, (i, j) in zip(location_names, node_indices):
                if data_type == 'omon':
                    values = dataset.variables[var][t, :, i, j]
                    time_series_data[name].append([times[t]] + values.tolist())
                elif data_type == 'amon':
                    var_value = dataset.variables[var][t, i, j]
                    time_series_data[name].append([times[t], var_value])
        
        dataset.close()

    for name in location_names:
        if data_type == 'omon':
            df = pd.DataFrame(time_series_data[name], columns=['Date'] + [f'{var}_depth_{d}' for d in range(50)])
            df.to_csv(f'{name}_{var}_{ssp}_ts_alldepths.csv', index=False)
            print(f'The file {name}_{var}_{ssp}_ts_alldepths.csv has been written.')
        elif data_type == 'amon':
            df = pd.DataFrame(time_series_data[name], columns=['Date', var])
            df.to_csv(f'{name}_{var}_{ssp}_ts.csv', index=False)
            print(f'The file {name}_{var}_{ssp}_ts.csv has been written.')


if location_type == 'domain':
    sample_dataset = nc.Dataset(filepath, mode='r')
    if data_type == 'omon':
        lats = sample_dataset.variables['latitude'][:]
        lons = sample_dataset.variables['longitude'][:]
    elif data_type == 'amon':
        lats = sample_dataset.variables['lat'][:]
        lons = sample_dataset.variables['lon'][:]
    sample_dataset.close()

    lat_mask = (lats >= lat_min) & (lats <= lat_max)
    lon_mask = (lons >= lon_min) & (lons <= lon_max)
    unique_cells = {}
    for i in range(lats.shape[0]):
        for j in range(lons.shape[1]):
            if lat_mask[i, j] and lon_mask[i, j]:
                lat_lon_key = f'lat_{lats[i, j]:.2f}_lon_{lons[i, j]:.2f}'
                unique_cells[lat_lon_key] = (i, j)

    for key in unique_cells.keys():
        time_series_data[key] = []

    for filepath in filepaths:
        dataset = nc.Dataset(filepath, mode='r')
        time = dataset.variables['time'][:]
        time_units = dataset.variables['time'].units
        time_calendar = dataset.variables['time'].calendar
        times = nc.num2date(time, units=time_units, calendar=time_calendar)

        for t in range(len(times)):
            for key, (i, j) in unique_cells.items():
                if data_type == 'omon':
                    variable_value = dataset.variables[var][t, 0, i, j]   #Here I am specifically taking for the surface depth
                elif data_type == 'amon':
                    variable_value = dataset.variables[var][t, i, j]
                time_series_data[key].append([times[t], variable_value])
        
        dataset.close()

    combined_df = pd.DataFrame()
    for key in time_series_data.keys():
        df = pd.DataFrame(time_series_data[key], columns=['Date', key])
        combined_df = df if combined_df.empty else pd.merge(combined_df, df, on='Date')

    if data_type == 'omon':
        combined_df.to_csv(f'{location_type}_{var}_{ssp}_ts_surface.csv', index=False)
        print(f'The file {location_type}_{var}_{ssp}_ts_surface.csv has been written.')
    elif data_type == 'amon':
        combined_df.to_csv(f'{location_type}_{var}_{ssp}_ts.csv', index=False)
        print(f'The file {location_type}_{var}_{ssp}_ts.csv has been written')



The file FINO2_pr_126_ts.csv has been written.
The file FINO3_pr_126_ts.csv has been written.
The file Borssele_pr_126_ts.csv has been written.
The file Belwind_pr_126_ts.csv has been written.
The file Anholt_pr_126_ts.csv has been written.
The file Samso_pr_126_ts.csv has been written.


## Shapefile based data extraction from NetCDF files directly

The following snippets are for netCDF files from which you want to extract data based on a Shapefile. Please ensure the type of data you are using whether it has multiple dimensions or not. The demo script for PR (atmospheric variable, Precipitation Flux with 3 dimensions time,lat,lon) is added here for demonstration purpose. This script will give a timeseries for the cells which are extracted from the shapefile. The first column contains the date/time from start to end based on the nc file. The remaining columns represent each of the unique cells based on latitude and longitude, to be written to a csv in the same format as the previous snippet.

In [13]:
import xarray as xr
import geopandas as gpd
import rioxarray
import pandas as pd

# Load the netcdf
nc_file_path = r'H:/Semester 4/Draft Preparations/Integrating SHP with CMCCESM2 Code/pr_3hr_CMCC-ESM2_historical_r1i1p1f1_gn_200001010130-200412312230.nc'
ds = xr.open_dataset(nc_file_path)
nc_file_name = os.path.basename(nc_file_path).replace('.nc','')
parts = nc_file_name.split('_')
variable = parts[0]
frequency = parts[1]
scenario = parts[3]

# Load the shapefile
shapefile = r'H:/Semester 4/Draft Preparations/Integrating SHP with CMCCESM2 Code/merged_region_interest.shp'
shp_file_name = os.path.basename(shapefile).replace('.shp','')
sf = gpd.read_file(shapefile)

# Focus on the specific variable
pr = ds['pr']

# Set the spatial dimensions
pr = pr.rio.set_spatial_dims(x_dim="lon", y_dim="lat")
pr.rio.write_crs('epsg:4326', inplace=True)

# For shapefile:
sf.set_crs('epsg:4326', inplace=True, allow_override=True)
sf = sf.to_crs(pr.rio.crs)

# Clip the dataset using the shapefile
clipped_nc = pr.rio.clip(sf.geometry, sf.crs, drop=True)

# Convert the clipped data to a pandas DataFrame
clipped_df = clipped_nc.to_dataframe().reset_index()

# Remove 'spatial_ref' column
clipped_df = clipped_df.drop(columns=['spatial_ref'])

# Format the lat_lon column names
clipped_df['lat_lon'] = clipped_df.apply(lambda row: f"lat_{row['lat']:.2f}_lon_{row['lon']:.2f}", axis=1)

# Pivot the DataFrame
clipped_pivot = clipped_df.pivot(index='time', columns='lat_lon', values='pr')

# # Inspect the reshaped DataFrame
# print(clipped_pivot.head(20))

# Save the reshaped DataFrame to a CSV file
output_file_path = f'H:/Semester 4/Draft Preparations/Integrating SHP with CMCCESM2 Code/{shp_file_name}_{variable}_{frequency}_{scenario}_ts.csv'
clipped_pivot.to_csv(output_file_path)
